In [2]:
import pandas as pd

In [3]:
from pandas import ExcelWriter
from pandas import ExcelFile

In [4]:
import re

In [5]:
df_Quotes = pd.read_excel('C:\\Users\\slice\\NLP POC\\Input\\Quotes.xlsx')

In [6]:
df_Quotes

,Item No,Description,Count,Quote Sent
0,1,Graphite Stick Pencils – Black – b2,1000,1100
1,2,Graphite Stick Pencils – Red – b2,1500,1100
2,3,Charcoal Pencils – Black,2500,1100
3,4,Color Pencils – Yellow,3000,1100


In [7]:
df_Supplier = pd.read_excel('C:\\Users\\slice\\NLP POC\\Input\\Supplier.xlsx')

In [8]:
df_Supplier

,Supplier No,Supplier Name,Supplier Email
0,1,Abhijith,abhijith.codepixer@outlook.com
1,2,Shailendra,s.shailendravarma@gmail.com


In [9]:
writer = pd.ExcelWriter('C:\\Users\\slice\\NLP POC\\Input\\Received Quotes.xlsx',engine='openpyxl')

In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [11]:
# CHANGING THE RULES
from spacy.pipeline import SentenceSegmenter

def split_on_newlines(doc):
    start = 0 
    seen_newline = False
    for word in doc:
        if seen_newline:
            yield doc[start:word.i]
            start = word.i
            seen_newline = False
        elif word.text.startswith('\n'): # handles multiple occurrences
            seen_newline = True
    yield doc[start:]      # handles the last group of tokens


sbd = SentenceSegmenter(nlp.vocab, strategy=split_on_newlines)
nlp.add_pipe(sbd)

In [12]:
myfile = open('C:\\Users\\slice\\OneDrive\\Desktop\\pdf1.txt')

In [13]:
MyFile_STR=myfile.read()

In [14]:
myfile.seek(0)

0

In [15]:
doc=nlp(myfile.read())

In [16]:
MyFile_STR

'The cost of Graphite Stick Pencils – Black – b2  is $2000.05\nThe cost of Graphite Stick Pencils – Red – b2 – 1500 Numbers - $3000.\n$3000 is the cost of Charcoal Pencils – Black.\nColor Pencils – Yellow – 3000 is not available.\n\n\n\n\nRegards,\n\n\n\nAbhijith S M\n\n\nabhijithsm94@outlook.com\nPhone: + 91 - 9940243959\n1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India\n    \n\n\n\n?\n\nFrom: Shailendra Varma Sagi <shailu.codepixer@outlook.com>\nSent: Friday, February 14, 2020 11:43 PM\nTo: Abhijith.Codepixer@outlook.com <Abhijith.Codepixer@outlook.com>\nSubject: SoftClouds – Quotation #1100 for Pencils \n \nQuote NO:#1100\n\nSupplier Name:Abhijith\n\nThanks for being a supplier for us. We would like to get a quote from you for the following items:\n?\nGraphite Stick Pencils – Black – b2 – 1000 Numbers?\n?\nGraphite Stick Pencils – Red – b2 – 1500 Numbers?\n?\nCharcoal Pencils – Black – 2500?\n?\nColor Pencils – Yellow – 3000?\n?\nPleas

In [17]:
len(MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>'))

2

In [18]:
MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[0]

'The cost of Graphite Stick Pencils – Black – b2  is $2000.05\nThe cost of Graphite Stick Pencils – Red – b2 – 1500 Numbers - $3000.\n$3000 is the cost of Charcoal Pencils – Black.\nColor Pencils – Yellow – 3000 is not available.\n\n\n\n\nRegards,\n\n\n\nAbhijith S M\n\n\nabhijithsm94@outlook.com\nPhone: + 91 - 9940243959\n1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India\n    \n\n\n\n?\n\nFrom: '

In [19]:
 x=re.search(r'\nQuote NO:#[\d]{4}\n',MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[1]).group()

In [20]:
Quote=re.search(r'[\d]{4}',x).group()

In [21]:
Quote

'1100'

In [22]:
 x=re.search(r'\nSupplier Name:[\w{8,16}]*\n',MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[1]).group()

In [23]:
supplier=x.replace('\nSupplier Name:','').replace('\n','')

In [24]:
supplier

'Abhijith'

In [25]:
df_Quotes = df_Quotes[ df_Quotes['Quote Sent'] == int(Quote) ]


In [26]:
df_Quotes

,Item No,Description,Count,Quote Sent
0,1,Graphite Stick Pencils – Black – b2,1000,1100
1,2,Graphite Stick Pencils – Red – b2,1500,1100
2,3,Charcoal Pencils – Black,2500,1100
3,4,Color Pencils – Yellow,3000,1100


In [27]:
QuoteRegex=''
for index,row in df_Quotes.iterrows():
    if QuoteRegex=='':
        QuoteRegex=row['Description']
    else:
        QuoteRegex=QuoteRegex +'|'+ row['Description']

In [28]:
QuoteRegex

'Graphite Stick Pencils – Black – b2|Graphite Stick Pencils – Red – b2|Charcoal Pencils – Black|Color Pencils – Yellow'

In [29]:
Output_df = pd.DataFrame(columns=['Supplier Name', 'QuoteID', 'Item Name','Quote Sent'])


In [30]:
Output_df

,Supplier Name,QuoteID,Item Name,Quote Sent


In [31]:
doc2=nlp(MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[0])

In [32]:
for sent in doc2.sents:
    #print(sent.text)
    #print(re.findall(QuoteRegex,sent.text))
    if len(re.findall(QuoteRegex,sent.text))==0:
        pass
       #print("Not Found")
    else:
        if len(re.findall(QuoteRegex,sent.text))==1 and len(re.findall(r'[£,$][1-9][\.\d]*(,\d+)?',sent.text))==1:
            if re.search(r'[£,$][1-9][\.\d]*(,\d+)?',sent.text).group()[-1]=='.':
                Quote_sent=re.search(r'[£,$][1-9][\.\d]*(,\d+)?',sent.text).group()[0:-1]
            else:
                Quote_sent=re.search(r'[£,$][1-9][\.\d]*(,\d+)?',sent.text).group()
                
            new_row = {'Supplier Name':supplier, 'QuoteID':Quote, 'Item Name':re.findall(QuoteRegex,sent.text)[0],'Quote Sent':Quote_sent}
            Output_df=Output_df.append(new_row,ignore_index=True)
            #print(re.findall(QuoteRegex,sent.text)[0]+'     '+ re.search(r'[£,$][1-9][\.\d]*(,\d+)?',sent.text).group())
        else:
            print('Something went wrong')
            
Output_df            


Something went wrong


,Supplier Name,QuoteID,Item Name,Quote Sent
0,Abhijith,1100,Graphite Stick Pencils – Black – b2,$2000.05
1,Abhijith,1100,Graphite Stick Pencils – Red – b2,$3000
2,Abhijith,1100,Charcoal Pencils – Black,$3000


In [33]:
Output_df.to_excel(writer, index=False)
Output_df.to_excel(writer, startrow=len(Output_df)+2, index=False)
writer.save()